In [1]:
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import coo_array
from pathlib import Path
import networkx as nx
import pandas as pd
import numpy as np

# TODO 
- May need to find a better way of incorporating # of transactions
- Method 2
- CHANGE EDGE DATASETS TO USE ACTUAL DATA

# 1. Appending Node Scores
1. Calculate pagerank with edge score only
2. Scale pageranks by min and max pageranks
3. Add node scores to pageranks

In [2]:
DATAPATH = Path('../data/processed')

n_df = pd.read_parquet(DATAPATH / 'node_score.parquet')
e_df = pd.read_parquet(DATAPATH / 'temp_edge_score.parquet')

In [3]:
# Combine identical edges
e_df = e_df.groupby(['cust_id_sender', 'cust_id_receiver'], as_index=False)['score'].sum()

# Construct graph
G = nx.from_pandas_edgelist(e_df, 'cust_id_sender', 'cust_id_receiver', edge_attr='score', create_using=nx.DiGraph())

# Pagerank
pr = nx.pagerank(G, weight='score')

In [4]:
# Scaling
pr_df = pd.DataFrame.from_dict(pr, orient='index').reset_index()
pr_df = pr_df.rename(columns={'index': 'cust_id', 0:'score'})

max_score = pr_df['score'].max()
min_score = pr_df['score'].min()
pr_df['score'] = (pr_df['score'] - min_score)/(max_score - min_score)

# Adding node scores
pr_df = pr_df.merge(n_df, on='cust_id', how='left')
pr_df['score_y'] = pr_df['score_y'].fillna(0)
pr_df['score'] = pr_df['score_x']+pr_df['score_y']
pr_df = pr_df[['cust_id', 'score']]

# Exporting
pr_df.to_parquet(DATAPATH / 'pagerank_1.parquet')
pr_df.sample(3)

,cust_id,score
163925,CUST65878723,0.000000
95717,CUST37378780,0.187013
222858,CUST43570661,0.186988


# 2. Some better way using networkx

# 3. Zhang et al. Method
This method uses the pagerank technique outlined in [Zhang *et al.,* 2022](https://arxiv.org/abs/2104.02764). 

Mainly: 

Let $M$ be a transition matrix with: 
$$m_{ij} = \begin{cases}
    \theta w_{ji}/s_j^{out} + (1-\theta)a_{ji}/d_j^{out} && \texttt{if}\; d_j^{out}\neq 0\\
    \beta_{i}/\sum_{i\in V}\beta_i && \texttt{if}\; d_j^{out}=0
\end{cases}$$
Where: 
- $\theta$ is a tunable parameter that represents how important weights should be in the pagerank alg
- $w_{ji}$ is the weight of an edge from node $j \rightarrow i$
- $s_j^{out} = \sum_{v \in V|j \rightarrow v}w_{jv}$ is the "strength" of outgoing edges from node $j$
- $a_{ji}$ = $1 \:\texttt{if}\: j \rightarrow i \:\texttt{else}\: 0$
- $d_j^{out} = \sum_{v \in V|j \rightarrow v}a_{jv}$
- $\beta_i$ is the node importance score
  


Then the pagerank can be calculated with power iterations on
$$P=\gamma MP + (1-\gamma)\boldsymbol{\beta}/||\boldsymbol{\beta}||_1$$

Where:
- $1-\gamma$ is a tunable parameter representing the probability of restarting a random walk (typically 0.8-0.9)

---
*note the other formulation in the paper results in a dense transition matrix that is nxn.... too big*

*note that for $m_{ij}$, $i$ is the target and $j$ is the source*

In [5]:
THETA = 0.5
GAMMA = 0.85

DATAPATH = Path('../data/processed')

n_df = pd.read_parquet(DATAPATH / 'node_score.parquet')
e_df = pd.read_parquet(DATAPATH / 'temp_edge_score.parquet')

## Constructing $\boldsymbol{\beta}/||\boldsymbol{\beta}||_1$

In [6]:
# Get all nodes in the graph
node_list = []
node_list.extend(e_df.cust_id_sender.tolist() + e_df.cust_id_receiver.tolist())
node_list = list(set(node_list)) #hack to remove duplicates

In [7]:
# Construct B
b_df = pd.DataFrame(data={'cust_id':node_list})
b_df = b_df.merge(n_df, on='cust_id', how='left')
b_df = b_df.rename(columns={'score':'b_i'})
b_df = b_df.fillna(0)

one_norm = abs(b_df['b_i']).sum()

b_df['b_i'] = b_df['b_i'] / one_norm

b_df.sort_values('b_i', ascending=False).head(3)

,cust_id,b_i
166135,CUST75045024,0.000094
3588,CUST18423426,0.000094
49033,CUST40682551,0.000094


## Node Encoding
This section encodes the cust_ids ordinally.

In [8]:
le = LabelEncoder()
le.fit(node_list)

e_df['cust_id_sender'] = le.transform(e_df['cust_id_sender'])
e_df['cust_id_receiver'] = le.transform(e_df['cust_id_receiver'])

b_df['cust_id'] = le.transform(b_df['cust_id'])

e_df.sample(3)

,cust_id_sender,cust_id_receiver,trxn_id,trxn_type,score
303305,155360,198782,VBTQ98493205,wire,0.616436
86917,163771,180151,LZBD31039644,wire,0.005909
354983,117751,239808,EKTS47382358,wire,0.595851


## Constructing $M$
*note that for $m_{ij}$, $i$ is the target node and $j$ is the source node*

In [9]:
# Combine identical edges by adding weights (multidigraph -> digraph)
e_df = e_df[['cust_id_sender', 'cust_id_receiver', 'score']].copy()
e_df = e_df.groupby(['cust_id_sender', 'cust_id_receiver'], as_index=False).sum()

In [10]:
#Calculate s_j
s_df = e_df.groupby(['cust_id_sender'], as_index=False)['score'].sum()
s_df = s_df.rename(columns={'score':'s_j'})
e_df = e_df.merge(s_df, on='cust_id_sender', how='left')

#Calculate d_j
d_df = e_df.groupby(['cust_id_sender'], as_index=False)['score'].count()
d_df = d_df.rename(columns={'score':'d_j'})
e_df = e_df.merge(d_df, on='cust_id_sender', how='left')

#Merge b_i
e_df = e_df.merge(b_df, how='left', left_on='cust_id_receiver', right_on='cust_id')
e_df.sample(3)

,cust_id_sender,cust_id_receiver,score,s_j,d_j,cust_id,b_i
126241,66044,110619,0.980562,5.275213,8,110619,0.0
258798,136051,236557,0.135476,4.750757,10,236557,0.0
296414,155966,114781,0.434054,1.560070,3,114781,0.0


In [11]:
def calc_m(r): 
    if r.d_j == 0: 
        m = b_i
    else:
        m = THETA*r.score/r.s_j + (1-THETA)/r.d_j
    return m

#calculate m_ij for sender j, receiver i
e_df['m'] = e_df.apply(lambda r: calc_m(r), axis=1)
e_df.sample(3)

,cust_id_sender,cust_id_receiver,score,s_j,d_j,cust_id,b_i,m
249647,131112,14911,0.995885,3.911437,7,14911,0.000047,0.198733
135770,71179,106430,0.916023,3.612774,4,106430,0.000031,0.251776
38932,20136,238025,0.890713,4.612946,9,238025,0.000000,0.152101


## Pagerank

In [12]:
i = e_df['cust_id_receiver'].values
j = e_df['cust_id_sender'].values
m = e_df['m'].values

N = b_df.shape[0]
B = b_df.sort_values('cust_id')['b_i'].values

# Construct sparse matrix
M = coo_array((m,(i,j)), shape=(N,N))

# Verify that data is stored correctly
test_i = 243015
test_j = 1745
print(f'M at i={test_i} and j={test_j} is {M.tocsr()[test_i,test_j]}')

# Pagerank
P = np.full(N, 1/N)
max_diff = 1 #TODO implement
iter = 0 #TODO implement
for i in range(100):
    iter += 1
    P_int = GAMMA*M.dot(P) + (1-GAMMA)*B
    max_diff = np.absolute(P-P_int).max()
    P = P_int

# Scaling
P = (P - P.min())/(P.max() - P.min())

M at i=243015 and j=1745 is 0.1675836494070475


In [13]:
# Exporting
P_df = pd.DataFrame(data=P, columns=['score'])
P_df['cust_id'] = P_df.index
P_df['cust_id'] = le.inverse_transform(P_df['cust_id']) #transform back to actual IDS
P_df = P_df[['cust_id','score']]
P_df.to_parquet(DATAPATH / 'pagerank_3.parquet')